In [100]:
from bs4 import BeautifulSoup
import requests, csv, string, re
from csv import writer

abc = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
# abc = 'C'
abc = list(abc)
base_url = 'https://www.uktw.co.uk/archive/work/opera/'

In [101]:
def get_info(listings):
    rows = listings.find_all('tr')
    venue, from_date, to_date, company, composer, when_dates, town = ('','','','','','','')
    for row in rows:
        if venue == '':
            try:
                venue = row.find("b", string="Venue").parent.parent.text.replace('Venue', '')
            except:
                None
        if from_date == '':
            try:
                from_date = row.find("b", string="From").parent.parent.text.replace('From', '')
            except:
                None
        if to_date == '':
            try:
                to_date = row.find("b", string="To").parent.parent.text.replace('To', '')
            except:
                None      
        if company == '':
            try:
                company = row.find("b", text=re.compile("Company(.*)")).parent.parent.text.replace('Company', '').strip()
            except:
                None      
            try:
                company = row.find("b", text=re.compile("Producer(.*)")).parent.parent.text.replace('Producer', '').strip()
            except:
                None
            try:
                company = row.find("b", text=re.compile("Presented by(.*)")).parent.parent.text.replace('Presented by', '').strip()
            except:
                None                   
        if composer == '':
            try:
                composer = row.find("b", text=re.compile("Music(.*)")).parent.parent.text.replace('Music', '').strip()                
            except:
                None  
            try:
                composer = row.find("b", text=re.compile("Author(.*)")).parent.parent.text.replace('Author', '').strip()                
            except:
                None       
        if when_dates == '':
            try:
                when_dates = row.find("b", text=re.compile("When(.*)")).parent.parent.text.replace('When', '').strip()                
            except:
                None  
        if town == '':
            try:
                town = row.find("b", text=re.compile("Town(.*)")).parent.parent.text.replace('Town', '').strip()                
            except:
                None                  
    return venue, from_date, to_date, company, composer, when_dates, town

def get_more_info(listings):
    category = 'live'
    details = listings.find('h2', text='Production details')
    if details:
        if len(details.parent.find_all(text = re.compile('Broadcast'))) > 0:
            category = 'broadcast'
    return category

In [102]:
shows= []
for l in abc:
    url = base_url + l
    r  = requests.get(url)
    soup = BeautifulSoup(r.text, "html5lib")
    codes = soup.find_all('ul')[10].find_all('li')
    for code in codes:        
        opera_name = code.text.strip()
        link = code.find('a').get('href')
        r_show  = requests.get(link)
        listings = BeautifulSoup(r_show.text, "html5lib")
        for i in listings.find_all('i'):
            if len(i.find_all('a')) > 0:
                if i.find('a').get('href') != None:             
                    show = []
                    show.append(opera_name)
                    show.append(i.find('a').get('href'))
                    shows.append(show)

for show in shows:
    if show[1] != '':
        r  = requests.get(show[1])
        soup = BeautifulSoup(r.text, "html5lib")
        venue, from_date, to_date, company, composer, when_dates, town  = get_info(soup)
        category = get_more_info(soup)
        listing = []
        listing.append(venue)
        listing.append(town)
        listing.append(from_date)
        listing.append(to_date)
        listing.append(company)
        listing.append(composer)        
        listing.append(when_dates)         
        listing.append(category)

with open("uk_listings.csv", 'w') as outfile:
    mywriter = writer(outfile)
    mywriter.writerow(['work', 'venue', 'town', 'from', 'to', 'company', 'composer', 'dates', 'category'])    
    for city in mapped_cities:
        mywriter.writerow(city)           

MissingSchema: Invalid URL '/biog/Opera+Holland+Park': No schema supplied. Perhaps you meant http:///biog/Opera+Holland+Park?